In [1]:
import pandas as pd 
import numpy as np 

# Find similar players across leagues using clustering, or FP Mining. 
- Compute the players stats into vectors by normalizing the values 
- We get a feature_vector for each player, project that to an embedding space and perform clutersing 

In [2]:
pl_players = pd.read_csv('./data/premier_league_data_fb_ref.csv')
bundesliga_players = pd.read_csv('./data/bundesliga_data_fb_ref.csv')
la_liga_players = pd.read_csv('./data/la_liga_data_fb_ref.csv')
ligue_1_players = pd.read_csv('./data/ligue_1_data_fb_ref.csv')
serie_a_players = pd.read_csv('./data/serie_a_data_fb_ref.csv')

pl_season_data = pl_players.groupby(['Season'])
bundesliga_season_data = bundesliga_players.groupby(['Season'])
la_liga_season_data = la_liga_players.groupby(['Season'])
ligue_1_season_data = ligue_1_players.groupby(['Season'])
serie_a_season_data = serie_a_players.groupby(['Season'])


In [11]:
pl_chelsea_players = pl_players[pl_players['Squad'] == 'Chelsea']
pl_chelsea_players.head()

,Unnamed: 0,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
33,178,Andreas Christensen,dk DEN,DF,Chelsea,eng Premier League,21,1996.0,27,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,232,Antonio RÃ¼diger,de GER,DF,Chelsea,eng Premier League,24,1993.0,27,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,384,Callum Hudson-Odoi,eng ENG,FW,Chelsea,eng Premier League,16,2000.0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,408,Cesc FÃ bregas,es ESP,MF,Chelsea,eng Premier League,30,1987.0,32,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,422,Charly Musonda Jr.,be BEL,FW,Chelsea,eng Premier League,20,1996.0,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
keys = list(pl_chelsea_players.keys())


['Unnamed: 0',
 'Player',
 'Nation',
 'Pos',
 'Squad',
 'Comp',
 'Age',
 'Born',
 'MP',
 'Starts',
 'Min',
 '90s',
 'Gls',
 'Ast',
 'G-PK',
 'PK',
 'PKatt',
 'CrdY',
 'CrdR',
 'Gls.1']

# Normalize Numeric Columns

In [23]:
pl_chelsea_numeric_col = pl_chelsea_players.select_dtypes(include='number')
# pl_chelsea_numeric_col.head()
pl_chelsea_norm = (pl_chelsea_numeric_col - pl_chelsea_numeric_col.mean()) / (pl_chelsea_numeric_col.max() - pl_chelsea_numeric_col.min())
pl_chelsea_players[pl_chelsea_numeric_col.columns] = pl_chelsea_norm
pl_chelsea_players['Age'] = pl_players['Age']
pl_chelsea_players['Born'] = pl_players['Born']
pl_chelsea_players.fillna(0.0)
pl_chelsea_players.head()

/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_16897/3516348819.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_chelsea_players[pl_chelsea_numeric_col.columns] = pl_chelsea_norm
/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_16897/3516348819.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_chelsea_players['Age'] = pl_players['Age']
/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_16897/3516348819.py:6: SettingWithCopyWarning: 
A value is trying to be set on a c

,Unnamed: 0,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
33,-0.502876,Andreas Christensen,dk DEN,DF,Chelsea,eng Premier League,21,1996.0,0.255324,0.234450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,-0.498852,Antonio RÃ¼diger,de GER,DF,Chelsea,eng Premier League,24,1993.0,0.255324,0.287081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,-0.487525,Callum Hudson-Odoi,eng ENG,FW,Chelsea,eng Premier League,16,2000.0,-0.420352,-0.370813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,-0.485737,Cesc FÃ bregas,es ESP,MF,Chelsea,eng Premier League,30,1987.0,0.390459,0.287081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,-0.484694,Charly Musonda Jr.,be BEL,FW,Chelsea,eng Premier League,20,1996.0,-0.393325,-0.370813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
flattened_list_cheslea = pl_chelsea_players[:].values.tolist()
flattened_list_cheslea[0]
fv_chelsea = list()
for l in flattened_list_cheslea:
    k = l[1]
    v = l[8:]
    y = l[3]
    fv_chelsea.append((k, v, y))

fv_chelsea[0]


('Andreas Christensen',
 [0.2553235053235053,
  0.23444976076555027,
  0.24138723115761682,
  0.24220319909975083,
  -0.11931818181818182,
  -0.09191919191919191,
  -0.11565656565656565,
  -0.024891774891774892,
  -0.030303030303030307,
  -0.1621212121212121,
  -0.03787878787878788,
  -0.12230412230412228,
  -0.07828282828282832,
  -0.1312688286372497,
  -0.11636636636636637,
  -0.12741449583554845,
  -0.07296367934665805,
  -0.06264775413711585,
  -0.12362991618310766,
  -0.11359427609427611,
  -0.12260536398467432,
  -0.07128663686040736,
  -0.11590455452730901,
  -0.11624869383490073,
  -0.1126383596443477,
  'Matches',
  -0.07513661202185792,
  -0.14221311475409837,
  -0.24582291666666667,
  -0.09918032786885246,
  -0.14706504494976202,
  -0.08958333333333333,
  -0.2801176470588235,
  -0.2610449735449736,
  -0.0625,
  -0.07427536231884056,
  -0.08368462138953942,
  -0.08921484037963762,
  0.2238605522284279,
  0.18275982970197846,
  0.22060560628776515,
  0.278139748454375,
  0.205